# Pytest tutorial

Need to install `pytest ipytest jupyter-core` to run this notebook

With `uv`, you can run this notebook with

```
uv tool run --with pytest --with ipytest --from jupyter jupyter lab
```

For further information on pytest look at https://docs.pytest.org/en/stable/how-to/index.html

## Simple tests

In [ ]:
%%writefile point.py
import math

class Point2D:
    def __init__(self, x, y):
        self.x = y
        self.y = x

    @property
    def phi(self):
        return math.atan2(self.x, self.y)

In [ ]:
%%writefile test_point.py

from math import pi
from point import Point2D

def test_zero():
    assert Point2D(1, 0).phi == 0

def test_pi2():
    assert Point2D(0, 1).phi == pi/2

def test_pi():
    assert Point2D(-1, 0).phi == pi

def test_minuspi2():
    assert Point2D(0, -1).phi == -pi/2

In [ ]:
!pytest test_point.py

If test filenames follow pattern `*_test.py` or `test_*.py` pytest discovers them automatically:

In [ ]:
!pytest

Now fix the flipped `x` and `y` assignment in `Point2D` and run again. Then fix the code to pass the tests again.

## Command line options

In [ ]:
%%writefile tests2.py

def test_print_something():
    print("printing something")

def test_print_something_else():
    print("printing something else")

def test_do_nothing():
    pass

In [ ]:
!pytest tests2.py

Use `-s` to deactivate output capturing (output will also be printed for failing tests)

In [ ]:
!pytest -s tests2.py

Use `-k` to filter tests

In [ ]:
!pytest -k nothing tests2.py

`-v` for verbose output

In [ ]:
!pytest -v tests2.py

And more ... (e.g. `--lf` to run last failed test, `--pdb` to drop into a debugger after an exception)

In [ ]:
!pytest --help

## Fixtures

From here on will use `ipytest` to directly run pytest in notebook cells

In [ ]:
import ipytest
ipytest.autoconfig()

In [ ]:
%%ipytest

import pytest

@pytest.fixture
def hans():
    return 42

def test_hans(hans):
    assert hans == 42

Things to try:
- make the test fail
- use `yield` instead of return and run some *set up* (preparation) before and some *tear down* (clean up) afterwards
- use a fixture within a fixture

### Built-in fixtures
See https://docs.pytest.org/en/stable/reference/fixtures.html#built-in-fixtures

E.g. `tmp_path` for per-test temporary directories

In [ ]:
%%ipytest

def test_in_temporary_directory(tmp_path):
    with open(tmp_path / "hans.txt", "w") as f:
        f.write("Bla!")

    with open(tmp_path / "hans.txt") as f:
        assert f.read() == "Bla!"

def test_another_temporary_directory(tmp_path):
    with open(tmp_path / "hans.txt") as f:
        assert f.read() == "Bla" # should fail

## Doctests

In [ ]:
%%writefile module_with_doctest.py

def atan2(y, x):
    """Return the arctangent of x,y
    >>> atan2(0, 0)
    0.0
    >>> atan2(1, 0)
    1.5707963267948966
    >>> atan2(0, -1)
    3.141592653589793
    >>> atan2(-1, 0)
    1.5707963267948966
    """
    import math
    return math.atan2(y, x)

In [ ]:
!pytest --doctest-modules module_with_doctest.py

## Parametrized tests

In [ ]:
from math import pi
from point import Point2D

In [ ]:
%%ipytest

def test_phi():
    for x, y, phi in [(1, 0, 0), (0, 1, pi/2), (-1, 0, pi), (0, -1, -pi/2)]:
        assert Point2D(x, y).phi == phi

In [ ]:
%%ipytest

@pytest.mark.parametrize("x,y,phi", [(1, 0, 0), (0, 1, pi/2), (-1, 0, pi), (0, -1, pi/2)])
def test_phi(x, y, phi):
    assert Point2D(x, y).phi == phi

More on marking: https://docs.pytest.org/en/stable/how-to/mark.html

## Test for an exception to occur

In [ ]:
%%ipytest

def test_raises():
    with pytest.raises(ZeroDivisionError):
        1 / 0

def some_function():
    raise ValueError("Wrong value for Hans")

def test_specific_value_error():
    with pytest.raises(ValueError, match=".*Hans.*"):
        some_function()

## Mocking and Monkeypatching

https://docs.pytest.org/en/7.1.x/how-to/monkeypatch.html

Temporarily modify global objects - e.g.
- overwrite defaults
- overwrite environment variables
- imitate interfaces that do things we may not want in a testing environment, e.g.
    - interact with a database
    - do a http request
    - ...

In [ ]:
import requests

def get_json(url):
    """Takes a URL, and returns the JSON."""
    r = requests.get(url)
    return r.json()

In [ ]:
%%ipytest

def test_get_json(monkeypatch):
    class MockResponse:
        def json(self):
            return {"answer": 42}

    # temporarily patch requests.get for this test
    monkeypatch.setattr(requests, "get", lambda url: MockResponse())

    result = get_json("http://dummy-url")
    assert result["answer"] == 42

def test_monty_python():
    # here requests.get will not be patched
    result = get_json("https://www.wikidata.org/w/api.php?action=wbgetentities&ids=Q16402&format=json")
    assert j["entities"]["Q16402"]["labels"]["en"]["value"] == "Monty Python"

## How to organize tests in a python package

### Directory structure
Typically in a `src` layout, have a directory structure like:

In [ ]:
!mkdir -p mypackage/src
!mkdir -p mypackage/tests

When writing your `pyproject.toml`, good idea to put `pytest` as an optional dependency for a "test" feature

In [ ]:
%%writefile mypackage/pyproject.toml

[project]
name = "mypackage"
version = "0.0"

[project.optional-dependencies]
test = ["pytest"]

In [ ]:
%%writefile mypackage/src/point.py

import math

class Point2D:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    @property
    def phi(self):
        return math.atan2(self.y, self.x)

In [ ]:
%%writefile mypackage/tests/test_phi.py
import pytest
from math import pi
from point import Point2D

@pytest.mark.parametrize("x,y,phi", [(1, 0, 0), (0, 1, pi/2), (-1, 0, pi), (0, -1, -pi/2)])
def test_phi(x, y, phi):
    assert Point2D(x, y).phi == phi

In [ ]:
!tree mypackage/

Now, in a terminal, install the package in a venv and execute the tests, e.g.

```bash
cd mypackage
python -m venv .venv
source .venv/bin/activate
pip install -e '.[test]'
pytest tests
```

### conftest.py
Put fixtures that are used by multiple test files here (not recommended to use imports!)

more info https://docs.pytest.org/en/7.1.x/how-to/fixtures.html#override-a-fixture-on-a-folder-conftest-level

### pytest.ini
https://docs.pytest.org/en/stable/reference/reference.html#configuration-options

Can e.g. put default command line arguments and test paths (e.g. restrict to only search in "tests")

In [ ]:
%%writefile mypackage/pytest.ini
[pytest]
addopts = -v
testpaths = tests

Alternative: put configuration in a `[tool.pytest.ini_options]` section in `pyproject.toml`